## Dataset Analysis And Assumption

12 csv files in relation to 12 different sites energy consumption between 2017-10-01 and 2018-09-30 with an interval of either 15 or 30 minutes. 1 file contains each site's state and 1 file contains holidays of each state.
Assumptions:
1. each site is independent of other sites
2. operating hours are indicated by relatively high energy consumption in aggregated intervals(2 30-min intervals or 4 15-min intervals)
3. operating hours do not apply to days when site is close i.e. public hoildays
4. operating hours are relatively fixed for each site

# Solution Overview

This solution uses S3 as data repsository, Lambda function as processing server with API Gateway as front end. An end user can call API gateway via any programming language then visualize and/or further manipulate returned data. User can either manually upload file into S3 or automate the process.

## Architecture

<img src="nmi.jpg">

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from datetime import datetime
import requests

#file in S3 that to be processed
#I've uploaded all 12 files in an S3 bucket,
#you can see different output by changing file variable to different files names
file = 'NMIA1.csv'

#API to activate lambda function, a single file is processed each call
#use loop for muli-files processing
url = 'https://cptnldpq25.execute-api.us-east-2.amazonaws.com/test/portfolio?file='+file

#return requests response contains aggregated hourly E and daily E, mean, std, and skewness of E in original file
#aggregated hourly E and daily E are loaded into respective pandas dataframe for visualisation and further manipulation
res = requests.get(url=url)
hours = res.json()['hours']
df_hours = pd.DataFrame(data=json.loads(hours))
df_hours.index = df_hours.index.astype(int)
df_hours.sort_index(inplace=True)
days = res.json()['days']
df_days = pd.DataFrame(data=json.loads(days))
#mean, std and skewness help understand the shape of original data
mean = res.json()['mean']
std = res.json()['std']
skew = res.json()['skew']

#average std/mean ratio in acrosss 12 provided file is 0.57
#cutoff value for operating hours is 1 std less than mean if std/mean ratio is less than or equal to 0.57
#or 1/2 std less than mean if std/mean ratio if greater than 0.57
#this is an educated guess approach without knowing business meaning of the data
if std/mean<=0.57:
    cutoff = mean - std
else:
    cutoff = mean - std/2
    
#visualisation of operating hours represented in green colour
x = df_hours.index.values
y = df_hours['interval'].values
xy = [(x[i],y[i]) for i in range(len(x))]
fig, ax = plt.subplots(figsize=(12,8))
for start, stop in zip(xy[:-1],xy[1:]):
    x_val, y_val = zip(start, stop)
    if (y_val[1]>cutoff):
        ax.plot(x_val, y_val, color='green')
    else:
        ax.plot(x_val, y_val, color='red')
ax.set(xlabel='Hours', ylabel='Interval E',title='Operating Hours')
ax.set_xticks(np.arange(24))
ax.grid()
plt.show()

In [ ]:
#join holidays and nmi_info table on State
#return site_holidays dataframe contains all holiday dates for each site
holidays = pd.read_csv('holidays.csv')
nmi_info = pd.read_csv('nmi_info.csv')
data = {}
for i, row in nmi_info.iterrows():
    site = row['Nmi']
    state = row['State']
    dates = holidays[holidays['State']==state]['LocalDate'].values
    data[site] = dates
site_holidays = pd.DataFrame(dict([(k,pd.Series(v)) for k, v in data.items()]))
site_holidays = site_holidays.apply(lambda x: pd.to_datetime(x),axis=1)
site_holidays = site_holidays.apply(lambda x: x.dt.date,axis=1)

In [ ]:
#convert df_days' index from timestamp to date for comparison with holiday dates in site_holidays
lst = []
for x in df_days.index:
    lst.append(datetime.fromtimestamp(int(x)//1000).date())
df_days.index = lst

#extract 20 days with least E value and check whether these are holidays for the site
#20 is an arbitrary number that can be defined by end user
least_20 = df_days.sort_values(by=['E']).head(20)
least_20['Holiday'] = 'TBC'
each_site_holidays = site_holidays[file.split('.')[0]].values
for date in least_20.index:
    if date in each_site_holidays:
        least_20['Holiday'].loc[date] = 'Yes'
    else:
        least_20['Holiday'].loc[date] = 'No'
least_20

In [ ]:
#scatter plot all the daily E values with holidays in red
fig,ax = plt.subplots(figsize=(12,8))
for i in range(len(df_days)):
    if df_days.index.values[i] in each_site_holidays:
        ax.scatter(x=df_days.index.values[i],y=df_days['E'].values[i],c='red')
    else:
        ax.scatter(x=df_days.index.values[i],y=df_days['E'].values[i],c='blue',alpha=0.3)
ax.grid()
plt.show()